In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("Recommending Music") \
    .config("spark.executor.instances", "3") \
    .config("spark.executor.memory", "1g") \
    .getOrCreate()

/opt/spark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
software.amazon.awssdk#s3 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3116d41d-5652-4f92-bf82-0801eef58aca;1.0
	confs: [default]
	found software.amazon.awssdk#s3;2.26.30 in central
	found software.amazon.awssdk#aws-xml-protocol;2.26.30 in central
	found software.amazon.awssdk#aws-query-protocol;2.26.30 in central
	found software.amazon.awssdk#protocol-core;2.26.30 in central
	found software.amazon.awssdk#sdk-core;2.26.30 in central
	found software.amazon.awssdk#annotations;2.26.30 in central
	found software.amazon.awssdk#http-client-spi;2.26.30 in central
	found software.amazon.awssdk#utils;2.26.30 in central
	found org.reactivestreams#reactive-streams;1.0.4 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found software.amazon.awssdk#metrics-spi;2.26.30 in central
	found software.amazon.

In [4]:
raw_user_artist_data = spark.read.text('s3a://raw/audioscrobbler_data/user_artist_data.txt')

In [5]:
raw_user_artist_data.show(5)

+-------------------+
|              value|
+-------------------+
|       1000002 1 55|
| 1000002 1000006 33|
|  1000002 1000007 8|
|1000002 1000009 144|
|1000002 1000010 314|
+-------------------+
only showing top 5 rows



In [6]:
raw_artist_data = spark.read.text('s3a://raw/audioscrobbler_data/artist_data.txt')

In [7]:
raw_artist_data.show(5)

+--------------------+
|               value|
+--------------------+
|1134999\t06Crazy ...|
|6821360\tPang Nak...|
|10113088\tTerfel,...|
|10151459\tThe Fla...|
|6826647\tBodensta...|
+--------------------+
only showing top 5 rows



In [8]:
raw_artist_alias = spark.read.text('s3a://raw/audioscrobbler_data/artist_alias.txt')

In [9]:
raw_artist_alias.show(5)

+-----------------+
|            value|
+-----------------+
| 1092764\t1000311|
| 1095122\t1000557|
| 6708070\t1007267|
|10088054\t1042317|
| 1195917\t1042317|
+-----------------+
only showing top 5 rows



## Preparando os Dados

In [10]:
from pyspark.sql.functions import col, split, min, max
from pyspark.sql.types import IntegerType, StringType

In [11]:
user_df_artist = raw_user_artist_data.withColumn('user', split(raw_user_artist_data['value'], ' ').getItem(0).cast(IntegerType()))

In [12]:
user_df_artist = user_df_artist.withColumn('artist', split(raw_user_artist_data['value'], ' ').getItem(1).cast(IntegerType()))

In [13]:
user_df_artist = user_df_artist.withColumn('count', split(raw_user_artist_data['value'], ' ').getItem(2).cast(IntegerType())).drop('value')

In [14]:
user_df_artist.select([min('user'), max('user'), min('artist'), max('artist')]).show()

+---------+---------+-----------+-----------+
|min(user)|max(user)|min(artist)|max(artist)|
+---------+---------+-----------+-----------+
|       90|  2443548|          1|   10794401|
+---------+---------+-----------+-----------+

